In [1]:
import geopandas as gpd
from shapely.geometry import Point, LineString, MultiPoint, MultiLineString
from shapely.ops import nearest_points
import pandas as pd
import glob
import os

#print(gdf_distritos[["ds_codigo", "ds_nome"]])

In [2]:
gdf_distritos = gpd.read_file(os.path.join(os.pardir, "download", "SIRGAS_SHP_distrito", "SIRGAS_SHP_distrito", "SIRGAS_SHP_distrito_polygon.shp"))

In [3]:
gdf_vias = gpd.read_file(os.path.join(os.pardir, "download", "SIRGAS_SHP_logradouronbl", "SIRGAS_SHP_logradouronbl", "SIRGAS_SHP_logradouronbl.shp"))

In [4]:
gdf_ferrovias = gpd.read_file(os.path.join(os.pardir, "download", "SIRGAS_SHP_ferroviamdc", "SIRGAS_SHP_ferroviamdc.shp"))

In [15]:
gdf_ferrovias['lg_codlog'] = 'ferrovia'
gdf_vias = gpd.GeoDataFrame(pd.concat([gdf_vias, gdf_ferrovias]).reset_index().drop(columns=['index', 'level_0']))

In [16]:
gdf_vias

,lg_codlog,lg_tipo,lg_titulo,lg_prep,lg_nome,lg_or_deno,lg_ind_ofi,lg_ini_par,lg_fim_par,lg_ini_imp,lg_fim_imp,lg_ordem,lg_id,lg_seg_id,lg_or_geom,geometry,fm_id
0,346438,R,None,None,GAMA BARROS,None,OFICIAL,28.0,110.0,29.0,111.0,2.0,64429.0,103294.0,None,"LINESTRING (340737.769 7390143.226, 340795.619...",NaN
1,283371,R,None,None,DEZOITO (J. PAULISTANO-JAR),None,OFICIAL,544.0,566.0,545.0,567.0,9.0,58117.0,259189.0,None,"LINESTRING (325401.339 7404855.962, 325385.258...",NaN
2,442992,AV,None,None,AIRTON PRETINI,None,OFICIAL,NaN,NaN,453.0,721.0,18.0,71934.0,305921.0,None,"LINESTRING (341219.899 7397215.689, 341243.447...",NaN
3,088617,R,None,DO,HORTO,None,OFICIAL,1254.0,1512.0,1255.0,1513.0,17.0,38555.0,121499.0,None,"LINESTRING (333620.846 7404427.630, 333620.899...",NaN
4,None,None,None,None,None,None,NAO OFICIAL,190.0,250.0,191.0,251.0,3.0,15408.0,258357.0,None,"LINESTRING (329586.427 7388972.035, 329596.797...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236094,ferrovia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (349893.073 7402276.835, 349900.972...",8013440.0
236095,ferrovia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (326063.715 7368044.988, 326061.732...",8013445.0
236096,ferrovia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (326437.256 7366973.780, 326437.188...",8013446.0
236097,ferrovia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (325750.927 7386265.923, 325739.022...",8013447.0


In [17]:
#gdf_quadras = gpd.read_file('../download/SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')

In [18]:
def centroide_por_segmento(linha):
    segmentos = list(map(LineString, zip(linha.coords[:-1], linha.coords[1:])))
    centroides = list(map(lambda x: x.centroid, segmentos))
    return MultiPoint(centroides)

In [19]:
def segmentos(linha):
    segmentos = list(map(LineString, zip(linha.coords[:-1], linha.coords[1:])))
    # centroides = list(map(lambda x: x.centroid, segmentos))
    return MultiLineString(segmentos)

In [20]:
## Para cada distrito
folder = '../download/lotes'
for path in glob.iglob(f"{folder}/*09*"):
    cod_distrito = path.replace(f"{folder}\\SIRGAS_SHP_LOTES_", '')[0:2]
    gdf_distrito = gdf_distritos[gdf_distritos.ds_codigo == str(int(cod_distrito))]

    ## Recorta NBL apenas do Distrito considerando um buffer de 20m
    gdf_viario = gpd.clip(gdf_vias, gdf_distrito.buffer(30))

    ## Remove tipo de logradouro PA
    gdf_viario = gdf_viario.loc[gdf_viario.lg_tipo != 'PA']

    ## Dissolve a NBL por CODLOG
    gdf_viario = gdf_viario.dissolve(by='lg_codlog').reset_index()

    ## Abre o arquivo de lotes do distrito
    gdf_lotes = gpd.read_file(f"zip://{path}!{path.replace(folder, '').replace('.zip', '')[1:]}")

    ## Cria um viário unificado para projetar os pontos mais próximos
    gdf_viario_unified = gdf_viario.unary_union

    ## Remove os lotes do Tipo V que são sobrepostos por viário
    gdf_lotes_v_vias = gdf_lotes[gdf_lotes.lo_tp_lote == 'V'].geometry.intersects(gdf_viario_unified)
    gdf_lotes.drop(gdf_lotes_v_vias[gdf_lotes_v_vias].index, inplace=True)

    ## Remove os lotes do Tipo V que NÃO SÃO sobrepostos por viários e criar um segmento de linha com CODLOG provisório para conectar a testada
    if len(gdf_lotes.loc[gdf_lotes.lo_tp_lote == 'V', ['lo_tp_lote', 'lo_setor', 'lo_quadra', 'lo_lote']].values.tolist()) > 0:
        gdf_viario = gpd.GeoDataFrame(pd.concat([gdf_viario, gpd.GeoDataFrame(pd.Series(gdf_lotes.loc[gdf_lotes.lo_tp_lote == 'V', ['lo_tp_lote', 'lo_setor', 'lo_quadra', 'lo_lote']].values.tolist(), index=gdf_lotes[gdf_lotes.lo_tp_lote == 'V'].index, name='lg_codlog').str.join(''), geometry=gdf_lotes[gdf_lotes.lo_tp_lote == 'V'].buffer(-.5).boundary)]))
        gdf_viario_unified = gdf_viario.unary_union

    ## Removendo lotes tipo V
    gdf_lotes = gdf_lotes[gdf_lotes.lo_tp_lote != 'V']

    ## Dissolve o arquivo de lotes por Setor, Quadra, Tipo e talvez Subquadra
    gdf_lotes_dissolved = gdf_lotes.dissolve(by=['lo_setor', 'lo_quadra', 'lo_tp_quad']).reset_index()
    gdf_faces = gpd.GeoDataFrame([], geometry=gdf_lotes_dissolved.explode().exterior.droplevel(1).reset_index().drop(columns='index')[0])
    # gdf_faces = gpd.GeoDataFrame([], geometry=gdf_lotes_dissolved.boundary.explode().droplevel(0).reset_index().drop(columns='index')[0])

    ## Calcula-se o centroide de cada segmento de linha de cada quadra
    gdf_pontos_faces = gpd.GeoDataFrame(geometry=gdf_faces.geometry.apply(lambda x: centroide_por_segmento(x)))

    ## Cria um buffer de alguns centimetros para o lote para certificar que o ponto esteja inserido nele
    gdf_lotes_buffered = gpd.GeoDataFrame(gdf_lotes, geometry=gdf_lotes.buffer(.05))

    ## Relaciona cada centroide de segmento de linha ao lote
    gdf_pontos_lotes = gpd.sjoin(gdf_pontos_faces.reset_index().explode().droplevel(0).drop(columns='index'), gdf_lotes_buffered, how='left')
    
    ## Conecta-se o centroide de cada segmento de linha ao segmento de logradouro mais próximo
    gdf_pontos_lotes_copy = gdf_pontos_lotes.copy(deep=True)
    gdf_conexoes = gpd.GeoDataFrame(gdf_pontos_lotes_copy, \
        geometry=gdf_pontos_lotes_copy.geometry.apply(lambda x: \
            LineString(nearest_points(x, gdf_viario_unified))))

    ## Cria Buffer de 50cm do viario
    gdf_viario_buffered = gpd.GeoDataFrame(gdf_viario, geometry=gdf_viario.buffer(.10))

    ## ajuste de indice
    gdf_conexoes = gdf_conexoes.drop(columns=['index_right'])

    ## Cria conexão com a via
    gdf_conexoes_relacionadas = gpd.sjoin(gdf_conexoes, \
        gdf_viario_buffered[['lg_codlog', 'geometry']], how='left', op='intersects')

    ## Relaciona os pontos ao CODLOG
    gdf_pontos_lotes_buffered = gpd.GeoDataFrame(gdf_pontos_lotes, geometry=gdf_pontos_lotes.buffer(0.01))
    gdf_pontos_lote_buffered_relacionados = gpd.sjoin(gdf_pontos_lotes_buffered.drop(columns=['index_right']), gdf_conexoes_relacionadas[['lg_codlog', 'geometry']], how='left')

    ## Segmentos de faces de lotes
    gdf_faces_segmentos = gpd.GeoDataFrame(geometry=gdf_faces.geometry.apply(lambda x: segmentos(x)))
    gdf_faces_segmentos = gdf_faces_segmentos.explode().droplevel(0).reset_index().drop(columns=['index'])  

    # ## Relaciona-se essa conexão criada ao CODLOG e Lote
    gdf_testadas = gpd.sjoin(gdf_faces_segmentos, gdf_pontos_lote_buffered_relacionados.drop(columns=['index_right']), how='left', op='intersects')

    ## Dissolve-se os segmentos de linha das faces dos lotes em testadas por CODLOG agrupadas por Lote fiscal
    gdf_testadas_dissolved = gdf_testadas.dissolve(by=['lo_setor', 'lo_quadra', 'lo_lote', 'lo_condomi', 'lo_tp_lote', 'lg_codlog'], as_index=False)

    ## Ajustando o SRC
    gdf_testadas_dissolved = gdf_testadas_dissolved.set_crs('epsg:31983')

    ## Exporta os resultados para um arquivo contendo as testadas dos lotes do distrito selecionado
    distrito_prefix = path.replace(f"{folder}\\SIRGAS_SHP_LOTES_", '')[0:].replace('.zip', '').replace('_', '-').lower()
    gdf_testadas_dissolved.to_file(f"../resultados/{distrito_prefix}-testadas.gpkg", driver='GPKG')

    ## Grava Log
    print(distrito_prefix)


09-bom-retiro
